# Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm.auto import tqdm
import math

In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)

# Model

In [ ]:
# train에서 구현한 model 코드 그대로 사용하시면 됩니다.

# Get alphas_cumprod

In [ ]:
def get_beta_alpha_linear(beta_start=0.0001, beta_end=0.02, num_timesteps=1000):
    # DDPM 학습 및 샘플링에 쓰일 alpha, beta, alphas_cumprod 반환
    # Train에서 쓰인 함수와 정확히 같습니다.

    betas = np.linspace(beta_start, beta_end, num_timesteps, dtype=np.float32)
    betas = torch.tensor(betas)
    alphas = 1.0 - betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)

    return betas, alphas, alphas_cumprod

# Sampling

In [ ]:
# sample에서 구현한 model 코드 그대로 사용하시면 됩니다.

# Save Images

In [ ]:
def create_batch_images(count):
    with torch.no_grad():
        num_saved_images = 0
        # num_saved_images가 count보다 커지면 저장이 종료됩니다.
        while num_saved_images<=count:
            # B*3*64*64 크기의 tensor가 samples에 담기게 됩니다.
            samples = sample_ddim(model, shape=(batch_size, 3, image_size, image_size), alphas_cumprod=alphas_cumprod, device=device, ddim_steps=200, eta=0.0)
            # batch의 각 3*64*64 이미지를 PIL Image로 변환 후 저장됩니다.
            for j in range(batch_size):
                img = samples[j]
                img = img.numpy()
                img = np.transpose(img, (1, 2, 0))
                img = (img + 1.0) / 2.0
                img = np.clip(img, 0, 1)
                img = (img * 255).astype(np.uint8)
                pil_img = Image.fromarray(img)
                pil_img.save(
                    f'/content/fake/{num_saved_images+j}.jpg',
                    format='JPEG',
                    quality=85,
                    optimize=True,
                    progressive=True
                )
            num_saved_images += batch_size

In [ ]:
# 샘플링한 이미지들이 저장될 Colab 로컬 VM의 폴더를 생성합니다.
# 이후 폴더를 압축하여 gdrive에 옮깁니다.
!mkdir /content/fake

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
_, _, alphas_cumprod = get_beta_alpha_linear()
alphas_cumprod = alphas_cumprod.to(device)

# 학습할 때와 같은 batch_size 사용 권장
batch_size = 64
# image_size는 고정해주세요.
image_size = 64

# train할 때 저장한 모델 파라미터 pt파일 불러오기
model = DDPMModel().to(device)
model_path = ''
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
print(f'Model is loaded')

model.eval()

# 샘플링하고 싶은 이미지 수 (정확하게 3000장이 샘플링 되지는 않습니다.)
count = 3000
create_batch_images(count)

In [ ]:
# 샘플링한 이미지가 저장되어 있는 디렉토리로 이동합니다.
%cd /content/fake
# gdrive의 디렉토리에 압축파일을 저장합니다.
!zip -r /content/sample_directory/DDIM_sampled_images.zip .